CHOINKI NA SZACHOWNICY

Wizualizacja choinek została wygenerowana przy pomocy zbioru danych hugging face'a zawierającego otwierające pozycje szachów razem z ich nazwami i notacją fen oraz datasetu wygenerowanego przeze mnie, która była głównym silnikiem całej wizualizacji.

In [2]:
import pandas as pd
import fastparquet
import plotly.graph_objects as go
import numpy as np

In [3]:
PIECE_GEOMETRY = {
    'p': {'h': 0.5,  'w': 0.35, 'sym': 'circle',       'top_sz': 5}, # pion
    'r': {'h': 0.6,  'w': 0.60, 'sym': 'square',       'top_sz': 7}, # wieża
    'n': {'h': 0.7,  'w': 0.40, 'sym': 'diamond',      'top_sz': 7}, # skoczek
    'b': {'h': 0.8,  'w': 0.25, 'sym': 'diamond-open', 'top_sz': 7}, # goniec
    'q': {'h': 0.95, 'w': 0.55, 'sym': 'circle-open', 'top_sz': 12}, # hetman
    'k': {'h': 1.10, 'w': 0.50, 'sym': 'cross',       'top_sz': 10}, # król
}

COLORS = { 'white_tree': '#F0FFFF', 'black_tree': '#006400', 'white_top': 'silver', 'black_top': 'gold', 'trunk': '#3E2723', 'board_light': '#DEB887', 'board_dark': '#8B4513'}

df = pd.read_parquet("hf://datasets/Lichess/chess-openings/data/train-00000-of-00001.parquet", engine='fastparquet')

In [4]:
df

,eco-volume,eco,name,pgn,uci,epd,img.bytes,img.path
0,A,A00,Amar Opening,1. Nh3,g1h3,rnbqkbnr/pppppppp/8/8/8/7N/PPPPPPPP/RNBQKB1R b...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
1,A,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4,g1h3 d7d5 g2g3 e7e5 f2f4,rnbqkbnr/ppp2ppp/8/3pp3/5P2/6PN/PPPPP2P/RNBQKB...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
2,A,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...,g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4 e1g1 f...,rn1qkbnr/ppp2ppp/8/3p4/8/6PB/PPPPP3/RNBQ1RK1 b...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
3,A,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6,e2e3 e7e5 c2c4 d7d6 b1c3 b8c6 b2b3 g8f6,r1bqkb1r/ppp2ppp/2np1n2/4p3/2P5/1PN1P3/P2P1PPP...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
4,A,A00,Anderssen's Opening,1. a3,a2a3,rnbqkbnr/pppppppp/8/8/8/P7/1PPPPPPP/RNBQKBNR b...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
...,...,...,...,...,...,...,...,...
3622,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d2d4 g8f6 c2c4 g7g6 b1c3 f8g7 e2e4 d7d6 g1f3 e...,r1bq1rk1/ppp1npbp/3p1np1/3Pp3/2P1P3/2N5/PP2BPP...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
3623,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d2d4 g8f6 c2c4 g7g6 b1c3 f8g7 e2e4 d7d6 g1f3 e...,r1bq1rk1/pppn2bp/3p2n1/2PPp1p1/4Pp2/2N2P2/PP2B...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
3624,E,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d2d4 g8f6 c2c4 g7g6 b1c3 f8g7 e2e4 d7d6 g1f3 e...,r1bq1rk1/pppnnpbp/3p2p1/3Pp3/2P1P3/2N1B3/PP2BP...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None
3625,E,E99,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d2d4 g8f6 c2c4 g7g6 b1c3 f8g7 e2e4 d7d6 g1f3 e...,r1bq1rk1/pppnn1bp/3p2p1/3Ppp2/2P1P1P1/2N2P2/PP...,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,None


In [6]:
def add_board_surface(fig):
    grid_x, grid_y = np.meshgrid(np.arange(8), np.arange(8))
    z_board = np.zeros(grid_x.shape)
    colors = (grid_x + grid_y) % 2

    fig.add_trace(go.Surface(
        x=grid_x, y=grid_y, z=z_board, surfacecolor=colors, colorscale=[[0, COLORS['board_light']], [1, COLORS['board_dark']]], showscale=False, opacity=1, lighting=dict(ambient=0.8, diffuse=0.8)))

def add_spiral_piece(fig, x, y, piece_char):
    p_type = piece_char.lower()
    is_white = piece_char.isupper()
    cfg = PIECE_GEOMETRY[p_type]
    tree_color = COLORS['white_tree'] if is_white else COLORS['black_tree']
    top_color = COLORS['white_top'] if is_white else COLORS['black_top']
    team_name = "White" if is_white else "Black"
    full_name = f"{team_name} {p_type.upper()}"
    trunk_h = 0.1
    total_h = trunk_h + cfg['h']
    N = 60

    z_spiral = np.linspace(trunk_h, total_h, N)
    if p_type == 'r':
         r_taper = (total_h - z_spiral)**0.6 * cfg['w']
    else:
         r_taper = (total_h - z_spiral) * cfg['w']

    theta = np.linspace(0, 6 * 2 * np.pi, N)

    x_spiral = x + r_taper * np.cos(theta)
    y_spiral = y + r_taper * np.sin(theta)

    fig.add_trace(go.Scatter3d(x=[x, x], y=[y, y], z=[0, trunk_h], mode='lines', line=dict(color=COLORS['trunk'], width=5), hoverinfo='skip'))
    fig.add_trace(go.Scatter3d(x=x_spiral, y=y_spiral, z=z_spiral, mode='lines', line=dict(color=tree_color, width=3), text=full_name, hoverinfo='text', name=full_name, showlegend=False))
    fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[total_h], mode='markers', marker=dict(size=cfg['top_sz'], color=top_color, symbol=cfg['sym'], line=dict(width=1, color='black')), hoverinfo='skip'))

def visualize_chess_spirals(fen, title):
    fig = go.Figure()
    add_board_surface(fig)
    rows = fen.split(' ')[0].split('/')
    for y_idx, row in enumerate(rows):
        y_pos = 7 - y_idx
        x_pos = 0
        for char in row:
            if char.isdigit():
                x_pos += int(char)
            else:
                add_spiral_piece(fig, x_pos, y_pos, char)
                x_pos += 1

    axis_template = dict(showgrid=False, zeroline=False, showbackground=False, ticks='', showticklabels=True)
    fig.update_layout(
        title=title, template="plotly_white",
        showlegend=False,
        scene=dict(
            xaxis=dict(title='', **axis_template, tickvals=list(range(8)), ticktext=list("ABCDEFGH")), yaxis=dict(title='', **axis_template, tickvals=list(range(8)), ticktext=list("12345678")), zaxis=dict(visible=False, range=[0, 2.5]), aspectmode='manual', aspectratio=dict(x=1, y=1, z=0.5), camera=dict(eye=dict(x=0, y=-1.6, z=1.6))), margin=dict(l=0, r=0, b=0, t=40))
    fig.show()

In [36]:
visualize_chess_spirals(df["epd"][5],df["name"][5])

In [8]:
visualize_chess_spirals(df["epd"][73],df["name"][73])

ciekawy dataset - świąteczna szachownica

In [29]:
import pandas as pd

# Wczytanie danych
pozycje = pd.read_csv('pozycje.csv')

In [30]:
pozycje.head()

,Nazwa,FEN
0,Choinka,8/4p3/3p1p2/2p3p2/1p5p/p7p/4P3/4K3 w - - 0 1
1,Świeczka,4K3/4P3/4P3/4P3/4P3/4P3/3PPP3/8 w - - 0 1
2,Dzwonek,4p3/3ppp2/3ppp2/2ppppp/8/8/8/8 w - - 0 1


In [31]:
visualize_chess_spirals(pozycje["FEN"][0],pozycje["Nazwa"][0])

In [32]:
visualize_chess_spirals(pozycje["FEN"][1],pozycje["Nazwa"][1])

In [33]:
visualize_chess_spirals(pozycje["FEN"][2],pozycje["Nazwa"][2])